<a href="https://colab.research.google.com/github/ikucherevsky/Brandefine/blob/main/Train_spacy_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy==2.3.4

In [ ]:
import pandas as pd 
import spacy
import pickle
spacy.require_gpu()
print("GPU for spaCy is ready")
from tqdm import tqdm
import random
import re
import warnings
import os
warnings.filterwarnings("ignore", category=UserWarning)
from spacy.gold import GoldParse
from spacy.scorer import Scorer
from spacy.matcher import PhraseMatcher

GPU for spaCy is ready


In [ ]:
!pip install pymorphy2
import pymorphy2
print('pymorphy2 succesfully installed')

In [ ]:
#load processed data
with open('/content/drive/MyDrive/dataset/TRAINING_DATA.pkl', 'rb') as input:
    TRAINING_DATA = pickle.load(input)

In [ ]:
# Кастомный токенизатор, для токенизация слов, написанных через дефис
# как единого токена, в остальном - аналогичен токенизатору по умолчанию
from spacy.tokenizer import Tokenizer
from spacy.util import compile_prefix_regex, compile_infix_regex, compile_suffix_regex

def custom_tokenizer(nlp):
    infix_re = re.compile(r'''[.\,\?\:\;\...\‘\’\`\“\”\"\'~]''')
    prefix_re = compile_prefix_regex(nlp.Defaults.prefixes)
    suffix_re = compile_suffix_regex(nlp.Defaults.suffixes)

    return Tokenizer(nlp.vocab, prefix_search=prefix_re.search,
                                suffix_search=suffix_re.search,
                                infix_finditer=infix_re.finditer,
                                token_match=None)

In [ ]:
# multi-model training
# Аналогично тренируем вторую модель, на TRAINING_DATA[450000:900000]
nlp = spacy.blank('ru')
ner = nlp.create_pipe('ner')
nlp.add_pipe(ner)
ner.add_label('BRAND')
ner.add_label('PRODUCT')
nlp.tokenizer = custom_tokenizer(nlp)

BATCH_SIZE = 1000
DATA = TRAINING_DATA[:450000]
BATCH_SIZE = 1000
nlp.begin_training()
for it in range(15):
    random.shuffle(DATA)
    losses = {}
    for batch in tqdm(spacy.util.minibatch(DATA, size = BATCH_SIZE)):
        texts = [text for text,_ in batch]
        annotations = [annotation for _, annotation in batch]
        nlp.update(texts, annotations,drop = 0.15, losses = losses)
    print(it,losses)
nlp.to_disk('NER_WITH_PRODUCT_450000_1st_part')

450it [05:50,  1.28it/s]


0 {'ner': 332190.5123500824}


450it [05:49,  1.29it/s]


1 {'ner': 136115.6136112213}


450it [05:52,  1.28it/s]


...

12 {'ner': 90554.33896446228}


450it [05:54,  1.27it/s]


13 {'ner': 89670.77554512024}


450it [06:00,  1.25it/s]

14 {'ner': 89219.70609474182}


In [ ]:
!zip -r ner_45000_1st_part.zip NER_WITH_PRODUCT_450000_1st_part

  adding: NER_WITH_PRODUCT_450000_1st_part/ (stored 0%)
  adding: NER_WITH_PRODUCT_450000_1st_part/vocab/ (stored 0%)
  adding: NER_WITH_PRODUCT_450000_1st_part/vocab/vectors (deflated 45%)
  adding: NER_WITH_PRODUCT_450000_1st_part/vocab/strings.json (deflated 82%)
  adding: NER_WITH_PRODUCT_450000_1st_part/vocab/key2row (stored 0%)
  adding: NER_WITH_PRODUCT_450000_1st_part/vocab/lookups.bin (stored 0%)
  adding: NER_WITH_PRODUCT_450000_1st_part/tokenizer (deflated 74%)
  adding: NER_WITH_PRODUCT_450000_1st_part/ner/ (stored 0%)
  adding: NER_WITH_PRODUCT_450000_1st_part/ner/model (deflated 7%)
  adding: NER_WITH_PRODUCT_450000_1st_part/ner/moves (deflated 56%)
  adding: NER_WITH_PRODUCT_450000_1st_part/ner/cfg (deflated 47%)
  adding: NER_WITH_PRODUCT_450000_1st_part/meta.json (deflated 48%)


## Evaluation

In [ ]:
from spacy.gold import GoldParse
from spacy.scorer import Scorer

def evaluate(ner_model, examples):
    scorer = Scorer()
    for input_, annot in examples:
        doc_gold_text = ner_model.make_doc(input_)
        gold = GoldParse(doc_gold_text, entities=annot['entities'])
        pred_value = ner_model(input_)
        scorer.score(pred_value, gold)
    return scorer.scores   

In [ ]:
evaluate(nlp,TRAINING_DATA[114000:124000])

Russian model ru2_combined_400ks_96


{'ents_f': 95.47040304053095,
 'ents_p': 95.44062606328684,
 'ents_per_type': {'BRAND': {'f': 92.16297786720322,
   'p': 92.06109938699628,
   'r': 92.2650820827878},
  'PRODUCT': {'f': 99.7463744553554,
   'p': 99.81777951321098,
   'r': 99.67507148427346}},
 'ents_r': 95.50019860409692,
 'las': 0.0,
 'las_per_type': {'': {'f': 0.0, 'p': 0.0, 'r': 0.0}},
 'tags_acc': 0.0,
 'textcat_score': 0.0,
 'textcats_per_cat': {},
 'token_acc': 100.0,
 'uas': 0.0}

In [ ]:
doc = nlp('напиток пивной  жатецкий гусь рубиновый   zatecky gus rubinovy  пастеризованный  бут  самара 0 48')
for ent in doc.ents:
    print(ent.text, ent.label_)

напиток PRODUCT
жатецкий гусь BRAND
